# Road Follower - Train Model


In this notebook we will train a neural network to take an input image, and output a set of x, y values corresponding to a target.

We will be using PyTorch deep learning framework to train ResNet18 neural network architecture model for road follower application.

このノートブックでは、入力画像を受け取り、ターゲットに対応するx, y値のセットを出力するニューラルネットワークを学習します。

PyTorch深層学習フレームワークを使用して、ResNet18ニューラルネットワークアーキテクチャモデルをロードフォロワーアプリケーション用に訓練する予定です。

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

### Download and extract data

Before you start, you should upload the ``road_following_<Date&Time>.zip`` file that you created in the ``data_collection.ipynb`` notebook on the robot. 

> If you're training on the JetBot you collected data on, you can skip this!

You should then extract this dataset by calling the command below:

In [3]:
# ↓のコマンドをターミナルから入力("!"は要らない). ファイル名は"road_following_<タイムスタンプの値>.zip"になっているので,ファイル名を直す事.
# !unzip -q road_following.zip

You should see a folder named ``dataset_all`` appear in the file browser.

### Create Dataset Instance

Here we create a custom ``torch.utils.data.Dataset`` implementation, which implements the ``__len__`` and ``__getitem__`` functions.  This class
is responsible for loading images and parsing the x, y values from the image filenames.  Because we implement the ``torch.utils.data.Dataset`` class,
we can use all of the torch data utilities :)

We hard coded some transformations (like color jitter) into our dataset.  We made random horizontal flips optional (in case you want to follow a non-symmetric path, like a road
where we need to 'stay right').  If it doesn't matter whether your robot follows some convention, you could enable flips to augment the dataset.

ここでは、カスタム ``torch.utils.data.Dataset`` の実装を作成し、 ``__len__`` と ``__getitem__`` 関数を実装しています。 このクラスは画像の読み込みと、画像ファイル名から x, y 値をパースする役割を担っています。 ここでは、 ``torch.utils.data.Dataset`` クラスを実装しているので、torch.utils.data.Dataset`` のすべての機能を利用することができます。

データセットにいくつかの変換をハードコードしています（色ずれなど）。 ランダムな水平方向の反転はオプションにしました。
水平方向のランダムフリップはオプションとしました（道路のように左右対称でない経路をたどる場合、「右に寄る」必要があるため）。 もし、あなたのロボットが何らかの慣習に従っているかどうかが重要でないなら、データセットを増やすために反転を有効にすることができる。

In [2]:
def get_x(path):
    """Gets the x value from the image filename"""
    return (float(int(path[3:6])) - 50.0) / 50.0

def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path[7:10])) - 50.0) / 50.0

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        x = float(get_x(os.path.basename(image_path)))
        y = float(get_y(os.path.basename(image_path)))
        
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=False)

### Split dataset into train and test sets


Once we read dataset, we will split data set in train and test sets. In this example we split train and test a 90%-10%. The test set will be used to verify the accuracy of the model we train.

データセットを読み込んだら、トレーニングセットとテストセットに分割します。  
この例では、トレーニングセットとテストセットを90%-10%に分割しています。テストセットは、学習したモデルの精度を検証するために使用されます。

In [3]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### Create data loaders to load data in batches


We use ``DataLoader`` class to load data in batches, shuffle data and allow using multi-subprocesses. In this example we use batch size of 64. Batch size will be based on memory available with your GPU and it can impact accuracy of the model.

``DataLoader`` クラスを使用する事でデータからバッチを取り出し、データをシャッフルし、複数のサブプロセスを使用できるようにします。  
この例では、64のバッチサイズを使用しています。バッチサイズはGPUで利用可能なメモリに依存し、モデルの精度に影響を与える可能性があります。

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### Define Neural Network Model 


We use ResNet-18 model available on PyTorch TorchVision. 

In a process called transfer learning, we can repurpose a pre-trained model (trained on millions of images) for a new task that has possibly much less data available.


More details on ResNet-18 : https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

More Details on Transfer Learning: https://www.youtube.com/watch?v=yofjFQddwHE 

ここでは、PyTorch TorchVisionで利用可能なResNet-18モデルを使用します。

転移学習と呼ばれるプロセスでは、事前に学習したモデル（数百万枚の画像で学習）を、利用可能なデータがはるかに少ない新しいタスクに再利用することができます。


ResNet-18の詳細: https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

転移学習の詳細: https://www.youtube.com/watch?v=yofjFQddwHE 

In [5]:
model = models.resnet18(pretrained=True)


ResNet model has fully connect (fc) final layer with 512 as ``in_features`` and we will be training for regression thus ``out_features`` as 1

Finally, we transfer our model for execution on the GPU

ResNetのモデルは最終層がfc(fully connect)で512がin_featuresで、回帰学習を行うのでout_featuresが1になる。

最後に、GPUで実行するためにモデルを転送します

In [6]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

### Train Regression:


We train for 70 epochs and save best model if the loss is reduced. 

``エポック数=70``で学習させ、損失が減少した場合に最適なモデルを保存する。

In [7]:
NUM_EPOCHS = 70
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)
    
    # 学習の進歩の表示
    if (epoch % 10) == 0:
        print('%d:' %(epoch),end='')  # 10の倍数のときに表示してわかりやすく
    print('%f, %f' % (train_loss, test_loss))  # printf("%f,%f",train_loss,test_loss)と同じ
    
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

0:0.674000, 4.075884
0.994902, 15.529646
0.529112, 7.117059
0.363284, 11.950798
0.113881, 7.538779
0.051340, 3.542394
0.052927, 2.390694
0.038607, 2.018709
0.038308, 1.458555
0.028485, 0.300918
10:0.028187, 0.247943
0.037752, 0.114871
0.025720, 0.142685
0.019077, 0.034020
0.020990, 0.019420
0.019403, 0.030323
0.015289, 0.100483
0.020974, 0.032053
0.016715, 0.021611
0.015682, 0.020184
20:0.018874, 0.024783
0.019231, 0.012621
0.014246, 0.017419
0.019842, 0.014694
0.012087, 0.016420
0.016436, 0.012167
0.010108, 0.034841
0.011386, 0.023710
0.021447, 0.013657
0.014532, 0.056721
30:0.022631, 0.020024
0.011501, 0.028916
0.013222, 0.019405
0.014606, 0.014857
0.012922, 0.013157
0.011013, 0.017894
0.007469, 0.016166
0.011692, 0.015192
0.009019, 0.023135
0.014596, 0.019646
40:0.010191, 0.029912
0.009251, 0.033519
0.006366, 0.008942
0.009123, 0.020934
0.008682, 0.018744
0.017016, 0.019579
0.010064, 0.023942
0.013666, 0.009351
0.011878, 0.022297
0.005841, 0.011489
50:0.008050, 0.013329
0.010711, 0.

Once the model is trained, it will generate ``best_steering_model_xy.pth`` file which you can use for inferencing in the live demo notebook.

If you trained on a different machine other than JetBot, you'll need to upload this to the JetBot to the ``road_following`` example folder.